In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from datetime import datetime

from whitebox.eval import WhiteBoxSensitivity
from whitebox.eval import WhiteBoxError

from sklearn.datasets import make_classification


In [2]:
x,y = make_classification(n_samples=10000,n_features=100)

In [3]:
df=pd.DataFrame(x)

In [4]:
columns=["column"+str(i) for i in df.columns ]
df.columns=columns
df['Dependant']=y

In [5]:

# set up y var
# set up some params
ydepend = 'Dependant'


df['column1bin'] = df['column1'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')
df['column5bin'] = df['column5'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1') 

string_categories = df.select_dtypes(include = ['O'])
# iterate over string categories
for cat in string_categories:
    df[cat] = pd.Categorical(df[cat]) 
    
df.select_dtypes(include = ['category'])
model_df = df.copy(deep = True)

# create dummies example using all categorical columns
dummies = pd.concat([pd.get_dummies(model_df.loc[:, col], prefix = col) for col in model_df.select_dtypes(include = ['category']).columns], axis = 1)
finaldf = pd.concat([model_df.select_dtypes(include = [np.number]), dummies], axis = 1)


In [6]:
clf = RandomForestClassifier()
# fit the model using the dummy dataframe
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])

start=datetime.now()
WB = WhiteBoxError(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1bin','column5bin'],
                    aggregate_func=np.mean,
                    verbose=None
                    )
WB.run()

WB.save('./CLASSIFICATION_scalability-Error-1.html')

print "RUN TIME----->", datetime.now()-start

ValueError: Wrong number of dimensions

In [ ]:

clf = RandomForestClassifier()
# fit the model using the dummy dataframe
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])

start=datetime.now()
print start
WB = WhiteBoxSensitivity(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1bin','column5bin'],
                    aggregate_func=np.mean,
                    verbose=None,
                     std_num=2
                    )
WB.run()

WB.save('./CLASSIFICATION_scalability-2.html')

print "RUN TIME----->", datetime.now()-start